<table style="width: 100%;" id="nb-header">
    <tr style="background-color: transparent;"><td>
        <img src="https://data-88e.github.io/assets/images/blue_text.png" width="250px" style="margin-left: 0;" />
    </td><td>
        <p style="text-align: right; font-size: 10pt;"><strong>Economic Models</strong>, EdX<br>
            Dr. Eric Van Dusen <br>
            Vaidehi Bulusu <br>
        Akhil Venkatesh <br>
</table>

# Lecture Notebook 8.2: Multiple Linear Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import patches
import statsmodels.api as sm
import warnings
warnings.simplefilter("ignore")
from datascience import *
%matplotlib inline
from sklearn.metrics import mean_squared_error
import ipywidgets as widgets
from ipywidgets import interact

# Multiple Linear Regression

In simple linear regression, we're only considering one independent variable. We're essentially assuming that this is the only variable that affects our dependent variable. Going back to the price and quantity demanded example, while price does have a huge effect on demand, there are other factors that also affect demand including income taxes, sales taxes, advertising, prices of related goods, etc. In the context of regression, because we are not including these variables in our model, they are called **omitted variables**.

Omitted variables cause 2 main problems:

- The regression parameters end up being inaccurate (biased) because of something called omitted variable bias. Your regression estimates for the slope and intercept are higher or lower than the actual values because of omitted variables (we are generally only concerned about the slope).


- They prevent you from inferring a causal association between the independent and dependent variables. In other words, you can't say that it's *because* of an increase in price that your quantity demanded decreased as there are so many other factors – like the change in the price of a related good – that could be causing the decrease in demand.

To try to eliminate omitted variable bias from our model, we take simple linear regression a step further: to multiple linear regression. In multiple linear regression, we are including more indepedent variables – variables we think are confounding variable that we've omitted – to reduce omitted variable bias.

Let's look at multiple linear regression in Python using a new dataset on earnings and various other factors (check out the [data description](https://wps.pearsoned.com/wps/media/objects/11422/11696965/empirical/empex_tb/CPS08_Description.pdf)).

In [2]:
cps = Table.read_table("CPS.csv")
cps.show(5)

year,ahe,bachelor,female,age
1992,11.1888,1,0,29
1992,10,1,0,33
1992,5.76923,0,0,30
1992,1.5625,0,0,32
1992,14.9573,1,0,31


Say we want to look at the relationship between age and earnings (the `ahe` column which is the average hourly earnings). We would expect whether or not a person has a bachelor's degree to be a confounding variable as those with a bachelor's degree typically earn more than those with only a high school degree. This is how we would do multiple linear regression:

In [3]:
x_2 = cps.select("bachelor", "age").values # This is how we include multiple independent variables in our model
y_2 = cps.column("ahe")
model_2 = sm.OLS(y_2, sm.add_constant(x_2))
result_2 = model_2.fit()
result_2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.153
Model:                            OLS   Adj. R-squared:                  0.153
Method:                 Least Squares   F-statistic:                     1381.
Date:                Tue, 25 Jul 2023   Prob (F-statistic):               0.00
Time:                        00:58:54   Log-Likelihood:                -54106.
No. Observations:               15316   AIC:                         1.082e+05
Df Residuals:                   15313   BIC:                         1.082e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -1.2711      0.708     -1.796      0.073      -2.659       0.116
x1             6.6590      0.135     49.333      0.000       6.394       6.924
x2             0.4621      0.024     19.551      0.000       0.416       0.508
==============================================================================
Omnibus:                     5161.829   Durbin-Watson:                   1.523
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            21800.024
Skew:                           1.619   Prob(JB):                         0.00
Kurtosis:                       7.866   Cond. No.                         316.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Dummy Variables

One type of variable commonly used in econometrics are dummy variables. These are variables that take a value of either 0 or 1 to indicate the presence of absence of a category. For example, take col: it takes the value of 1 to indicate that a person went to college and 0 to indicate that a person didn't go to college.

Let's do a regression of `ahe` on only `bachelor` (the dummy variable).

In [4]:
x_3 = cps.select("bachelor").values
y_3 = cps.column("ahe")
model_3 = sm.OLS(y_3, sm.add_constant(x_3))
result_3 = model_3.fit()
result_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.132
Method:                 Least Squares   F-statistic:                     2323.
Date:                Tue, 25 Jul 2023   Prob (F-statistic):               0.00
Time:                        00:58:54   Log-Likelihood:                -54294.
No. Observations:               15316   AIC:                         1.086e+05
Df Residuals:                   15314   BIC:                         1.086e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.4593      0.090    138.210      0.000      12.283      12.636
x1             6.5831      0.137     48.194      0.000       6.315       6.851
==============================================================================
Omnibus:                     5289.393   Durbin-Watson:                   1.538
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            22667.410
Skew:                           1.657   Prob(JB):                         0.00
Kurtosis:                       7.953   Cond. No.                         2.49
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The coefficient (or slope) on `bachelor` is this:

In [5]:
result_3.params[1]

6.583093534494762

This means that the people in our sample with a bachelor's degree earn around $6.583 more per hour than those with only a high school degree.

An interesting fact about dummy variables is that we can calculate this coefficient another way:

In [6]:
# Filter for bachelor = 1 and find mean earnings
b_1_mean = np.mean(cps.where("bachelor", 1).column("ahe"))

# Filter for bachelor = 0 and find mean earnings
b_0_mean = np.mean(cps.where("bachelor", 0).column("ahe"))

# Take the difference in the mean earnings
diff = b_1_mean - b_0_mean
diff

6.583093534494745

In [7]:
np.round(result_3.params[1], 5) == np.round(diff, 5)

True

These two values are pretty much the same! This is because the coefficient on a dummy x-variable is just equal to the difference of the mean of the y-variable when x = 1 and x = 0.

### `pd.get_dummies()`

We can convert categorical variables in our dataset to dummy variables using the pd.get_dummies() function. This function gives you a table showing the presence or absence of dummy variables for that category for each observation in the dataset. Here's an example of converting the year values to dummies. 1 indicates that the person was surveyed in that year (e.g. the first 4 people in the dataset were surveyed in 1992 according to the table below).

In [8]:
year = cps.column("year")
np.unique(year)

array([1992, 2008])

In [9]:
year_dummies = pd.get_dummies(year)
year_dummies.head()

,1992,2008
0,1,0
1,1,0
2,1,0
3,1,0
4,1,0


Let's do the same for the age variable. Take the first row as an example: since 1 is under 29, it means that that person is 29 years old.

In [10]:
age = cps.column("age")
np.unique(age)

array([25, 26, 27, 28, 29, 30, 31, 32, 33, 34])

In [11]:
age_dummies = pd.get_dummies(age)
age_dummies.head()

,25,26,27,28,29,30,31,32,33,34
0,0,0,0,0,1,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,1,0,0,0


### Dummy Variable Trap

One problem you may run into when using dummy variables is called the dummy variable trap. This happens when you include variables for all the values of the dummy variable: for example, you include col, which takes on a value of 1 if the person went to college, as well as notcol, which takes on a value of 1 if the person didn't go to college.

This causes redundancy as you can express one independent variable as a linear combination of another independent variable. In this case:

$$ notcol = 1 - col$$

This means that there is a perfect correlation between these two independent variables which reuslts in perfect multicollinearity. In general, multicollinearity occurs any time there is a high correlation between the independent variables in your model. It causes your regression estimates to be highly inaccurate.

Let's see what happens in Python when multicollinearity happens:

In [12]:
year_dummies["ahe"] = np.array(cps.column("ahe"))
year_dummies.head()

,1992,2008,ahe
0,1,0,11.188811
1,1,0,10.000000
2,1,0,5.769231
3,1,0,1.562500
4,1,0,14.957265


In [13]:
x_3 = year_dummies[[1992, 2008]] # We are including dummy variables for each value of year
y_3 = year_dummies["ahe"]
model_3 = sm.OLS(y_3, sm.add_constant(x_3))
result_3 = model_3.fit()
result_3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ahe   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.167
Method:                 Least Squares   F-statistic:                     3068.
Date:                Tue, 25 Jul 2023   Prob (F-statistic):               0.00
Time:                        00:58:55   Log-Likelihood:                -53977.
No. Observations:               15316   AIC:                         1.080e+05
Df Residuals:                   15314   BIC:                         1.080e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         10.2008      0.044    230.644      0.000      10.114      10.288
1992           1.4255      0.070     20.343      0.000       1.288       1.563
2008           8.7753      0.070    125.748      0.000       8.638       8.912
==============================================================================
Omnibus:                     4968.655   Durbin-Watson:                   1.764
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            20461.443
Skew:                           1.561   Prob(JB):                         0.00
Kurtosis:                       7.724   Cond. No.                     6.23e+15
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 5.92e-28. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

Python detected the multicollinearity and gave you a warning. A solution is to just drop one of the dummy variables.